# Notebook para poder validar los competitors de casi todos los clientes (menos YZA y los clients)

In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta
import re
import numpy as np

In [ ]:
doc_name = ""

In [ ]:
df = pd.read_csv(f"{doc_name}.csv", dtype = str, encoding='utf-8-sig')
df.shape 
df.columns = df.columns.str.lower()

In [ ]:
df["store id"].value_counts()

In [ ]:
LST_STORE_IDS = []

df_store_ids = set(df['store id'].unique())
missing_ids_in_df = list(set(LST_STORE_IDS) - df_store_ids)
missing_ids_in_df

In [ ]:
fecha_hoy = datetime.now().strftime("%Y-%m-%d")
conteo_fechas = df['date'].value_counts()

if fecha_hoy in conteo_fechas.index:
    print(f"✅ El archivo contiene la fecha de hoy")
else:
    print(f"❌ El archivo NO contiene la fecha de hoy")
conteo_fechas


In [ ]:
df[(df["final price"].isnull()) | (df["final price"] == "")]

In [ ]:
df = df.dropna(subset=["final price"])

In [ ]:
l = df[df["final price"] == df["final price"].max()]['url sku']
l.iloc[0]

In [ ]:
pd.set_option('display.max_columns', None)
df.sort_values('final price', ascending=False).head(10)

In [ ]:
df[df.duplicated(subset = ['sku', 'upc', "store id", "final price"])]["store id"].value_counts()
#df.drop_duplicates(inplace = True, subset = ['sku', 'upc', "store id", "final price"])

In [ ]:
df[(df["last_price"] != df["final price"]) & (df["last_price"].notna())]["canal"].value_counts()

In [ ]:
new_dir = f"{doc_name}.csv"

df.to_csv(new_dir, index=False, encoding="utf-8-sig")
new_dir

# Codigo para encontrar la cantidad de productos faltantes entre dos competitors

In [ ]:
nombre_df1 = ""
nombre_df2 = ""

df_actual = pd.read_csv(
    f".csv",
    dtype=str,
    encoding='utf-8'
)

df_anterior = pd.read_csv(
    f".csv",
    dtype=str,
    encoding='utf-8'
)

# Contar productos por store id en cada entrega
conteo_actual = df_actual.groupby("store id").size()
conteo_anterior = df_anterior.groupby("store id").size()

# Unir ambos conteos
comparacion = pd.DataFrame({
    "anterior": conteo_anterior,
    "actual": conteo_actual
}).fillna(0).astype(int)

# Calcular diferencia y porcentaje
comparacion["diferencia"] = comparacion["anterior"] - comparacion["actual"]
comparacion["porcentaje"] = (comparacion["diferencia"] / comparacion["anterior"] * 100).round(1)

# Filtrar solo donde faltaron productos
resultado = comparacion[comparacion["diferencia"] >=0].sort_values("porcentaje", ascending=False)
resultado = comparacion.sort_values("porcentaje", ascending=False)

print(resultado)

# Codigo para sacar los upcs nuevos y viejos de RFP

In [ ]:
import pandas as pd

# Cargar extracciones
df_new = pd.read_csv("csv", dtype=str, encoding='utf-8')
df_old = pd.read_csv(".csv", dtype=str, encoding='utf-8')

# Columnas para el merge (sin 'item') ""
MERGE_COLS = ["canal", "sku", "upc"]

# Columnas finales que queremos mantener
KEY_COLS = ["canal", "item", "sku", "upc"]

# Merge tipo outer solo por canal, sku, upc
df_compare = df_old.merge(
    df_new,
    on=MERGE_COLS,
    how="outer",
    indicator=True,
    suffixes=("_old", "_new")
)

# Crear columna flag basada en el indicador de merge
df_compare["flag"] = df_compare["_merge"].map({
    "right_only": 1,   # Solo en df_new → AGREGADO
    "left_only": 0,    # Solo en df_old → ELIMINADO
    "both": -1         # En ambos (se filtrará después)
})

# Filtrar solo los cambios (los que no están en ambos)
df_compare = df_compare[df_compare["_merge"] != "both"]

# Consolidar la columna 'item' (tomar de _new si existe, sino de _old)
df_compare["item"] = df_compare["item_new"].fillna(df_compare["item_old"])

# Mantener solo las columnas clave + flag
df_compare = df_compare[KEY_COLS + ["flag"]]

# Guardar
df_compare.to_csv("Entregas/26/nuevos_viejos_upcs_2026-01-12.csv", index=False)

# Mostrar resultado
print(df_compare)

# Opcional: Ver resumen de cambios
print(f"\nRegistros agregados (flag=1): {(df_compare['flag'] == 1).sum()}")
print(f"Registros eliminados (flag=0): {(df_compare['flag'] == 0).sum()}")